# agent 配置

## 基础配置

在 `Agent()` 中可以定义 `tool`，使用 `ModelSettings` 参数定义 `temperature`、`top_p` 等参数

In [5]:
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv
from litellm import completion

# 加载环境变量
load_dotenv()

# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

# api_key = os.getenv('DEEPSEEK_API_KEY')
# base_url = "https://api.deepseek.com/v1"
# chat_model = "deepseek/deepseek-chat"

In [ ]:
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    print(f"工具被调用了！接收到的city参数是：{city}")
    return f"The weather in {city} is sunny"

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather],  # 定义 tool
    model_settings=ModelSettings(temperature=0.2)
)

result = await Runner.run(agent, "上海天气怎么样？")  # "必须使用工具来获取信息" 如果不加这部分的 instructions，可能不会调用工具
print(result.final_output)

工具被调用了！接收到的city参数是：上海
上海城，天气晴朗，阳光明媚。


## 输出类型控制


pydantic

https://pydantic.com.cn/#pydantic_1

pydantic 允许铜鼓哦定义数据模型（继承自BaseModel），对传入的数据进行验证，确保数据符合预期的类型和结构

应用：
1. 格式化输出。铜鼓哦声明式的定义数据名和数据类型

In [ ]:
from pydantic import BaseModel

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

class Candidate(BaseModel):
    name: str
    gender: str
    location: str

agent = Agent(
    name="简历助手",
    instructions="根据要求的格式抽取相应的信息",
    model=llm,
    # 该参数会指示 agent 尝试生成符合 Candidate 这个模型定义的数据
    # 但受限于llm的指令遵循能力，以及约束的复杂性，不一定能百分百保证 agent 能实现格式化输出
    # 通过捕获异常，能定义没有格式化输出的位置，便于发现问题
    output_type=Candidate,
)

result = await Runner.run(agent, "我叫李婷，女，现居上海，想要应聘前台岗位。")
print(result.final_output)

## 上下文

上下文就是给大模型提供一些信息，但是和在 instruction 中的信息不同。instruction中的信息式固定的，始终需要的。而上下文可以让大模型**按需获取**


感觉对「上下文」这个概念不太理解  #TODO

In [ ]:
import asyncio
from dataclasses import dataclass
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# UserInfo 为上下文对象
@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    # 可以看到，所有的工具都可以访问到这个wrapper.context
    return f"User {wrapper.context.name} is 47 years old"

async def main():
    # 上下文的具体值
    user_info = UserInfo(name="John", uid=123)

    # agent 期望接收和处理的上下文类型为 UserInfo
    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

# if __name__ == "__main__":
    # asyncio.run(main())

await main()

# 为什么我运行代码输出结果为：The user is 47 years old.

## 动态指令

创建 agent 时需要提供 `instruction`，可以直接提供固定字符串，或者通过函数提供动态指令。

动态体现在哪里？

一般写法如下：

```python
# 定义动态指令函数
def dynamic_instructions(
    context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."


agent = Agent[UserContext](
    name="Triage agent",
    instructions=dynamic_instructions,
)
```

问题：
1. 为什么传入 `dynamic_instructions` 中的是 `context.context.name`？这个是什么？

In [ ]:
import asyncio
from dataclasses import dataclass
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


@dataclass
class PatInfo:
    # ip_type 是实例属性，不是静态属性
    # @dataclass 会为每个类型注解自动生成实例属性
    # 每个 PatInfo 实例都有自己独立的 ip_type 值
    ip_type: str  
    
    # self 是必需的，因为这是一个实例方法
    # self 允许方法访问实例的属性（如 self.ip_type）
    def advice(self) -> str:
        # 通过 self.ip_type 访问当前实例的 ip_type 属性
        # 如果 ip_type 是静态属性，就不需要通过 self 访问
        if self.ip_type=="发明":
            return "重点为用户讲解发明专利申请的实质审查要求"
        elif self.ip_type=="实用新型":
            return "重点为用户讲解实用新型专利申请的形式审查要求"


def dynamic_instructions(
    context: RunContextWrapper[PatInfo], agent: Agent[PatInfo]
) -> str:
    return f"用汉赋的句式{context.context.advice()}."


async def main():
    ip_info = PatInfo(ip_type="实用新型")

    agent = Agent[PatInfo](  
        name="Assistant",
        instructions=dynamic_instructions,  # 使用动态指令
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="我想申请专利",
        context=ip_info,
    )

    print(result.final_output)  


# if __name__ == "__main__":
#     asyncio.run(main())


await main()

In [4]:
@dataclass
class PatInfo:
    # ip_type 是实例属性，不是静态属性
    # @dataclass 会为每个类型注解自动生成实例属性
    # 每个 PatInfo 实例都有自己独立的 ip_type 值
    ip_type: str
    
    # self 是必需的，因为这是一个实例方法
    # self 允许方法访问实例的属性（如 self.ip_type）
    def advice(self) -> str:
        # 通过 self.ip_type 访问当前实例的 ip_type 属性
        # 如果 ip_type 是静态属性，就不需要通过 self 访问
        if self.ip_type=="发明":
            return "重点为用户讲解发明专利申请的实质审查要求"
        elif self.ip_type=="实用新型":
            return "重点为用户讲解实用新型专利申请的形式审查要求"

# 验证 ip_type 是实例属性的例子：
pat1 = PatInfo(ip_type="发明")
pat2 = PatInfo(ip_type="实用新型")
print(pat1.ip_type)  # 输出：发明
print(pat2.ip_type)  # 输出：实用新型
# 每个实例都有自己独立的 ip_type 值，证明它是实例属性

## 克隆

调用 agent 的 clone() 方法，快速基于 agnet 创建一个新的 agent

In [ ]:
pirate_agent = Agent(
    name="Pirate",
    instructions="Write like a pirate",
    model="o3-mini",
)

robot_agent = pirate_agent.clone(
    name="Robot",
    instructions="Write like a robot",
)